# Advanced Computer Vison Assignment

In [ ]:
import cv2
import numpy as np
from imutils.video import VideoStream

## Task 2 - Background Detection

## Task 3 - Object Detection

## Task 4 - Compare Object Detection Techniques

## Task 5 - Video Analysis